In [22]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import shape, mapping
import json
import geemap,ee
import folium
ee.Authenticate()
ee.Initialize()

In [97]:
iceberg1 = pd.read_csv('0003_06_01__05_12.csv', sep="&")
iceberg2 = pd.read_csv('0004_06_01__05_12.csv',sep="&")

In [98]:
iceberg1.head()

,fecha,similaridad,area,centroide,poligono,source,covers
0,2020-06-01,0.932003,1.037304e+08,"[-105.86525521394687, -74.4844911612252]","ee.FeatureCollection({\n ""functionInvocationV...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
1,2020-06-06,0.908518,1.049709e+08,"[-105.91986792088849, -74.44201168227734]","ee.FeatureCollection({\n ""functionInvocationV...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2,2020-06-11,0.898057,1.001686e+08,"[-106.17683309386798, -74.34499706400955]","ee.FeatureCollection({\n ""functionInvocationV...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
3,2020-06-16,0.936059,9.871248e+07,"[-106.06671230478825, -74.39021769184886]","ee.FeatureCollection({\n ""functionInvocationV...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
4,2020-06-16,0.936059,9.871248e+07,"[-106.06671230478825, -74.39021769184886]","ee.FeatureCollection({\n ""functionInvocationV...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."


In [99]:
iceberg1.drop_duplicates(inplace=True)
iceberg2.drop_duplicates(inplace=True)

In [100]:
iceberg1.reset_index(drop=True, inplace=True)
iceberg2.reset_index(drop=True, inplace=True)

In [102]:
iceberg1['centroide'] = iceberg1['centroide'].apply(lambda x: x.strip('][').split(', '))
iceberg2['centroide'] = iceberg2['centroide'].apply(lambda x: x.strip('][').split(', '))

In [103]:
iceberg1['centroide'] =  iceberg1['centroide'].apply(lambda x: list((float(x[0]), float(x[1]))))
iceberg2['centroide'] =  iceberg2['centroide'].apply(lambda x: list((float(x[0]), float(x[1]))))

In [104]:
iceberg1['fecha'] = pd.to_datetime(iceberg1['fecha'])
iceberg2['fecha'] = pd.to_datetime(iceberg2['fecha'])

In [105]:
iceberg1['lon'] = iceberg1['centroide'].apply(lambda x: x[0])
iceberg1['lat'] = iceberg1['centroide'].apply(lambda x: x[1])

In [124]:
iceberg2['lon'] = iceberg2['centroide'].apply(lambda x: x[0])
iceberg2['lat'] = iceberg2['centroide'].apply(lambda x: x[1])

In [120]:
iceberg1 = gpd.GeoDataFrame(
    iceberg1, geometry=gpd.points_from_xy(iceberg1['lon'], iceberg1['lat']), crs="EPSG:4326"
)

In [154]:
import re

def trans(contenido_celda):
    # Eliminar caracteres no válidos antes y después del JSON
    contenido_celda = contenido_celda.strip()

    match = re.search(r'(\{.*\})', contenido_celda, re.DOTALL)
    # Intentar cargar el JSON
    if match:
        json_str = match.group(1)
        
        # Intentar cargar el JSON
        try:
            json_dict  = json.loads(json_str)

            features = json_dict['functionInvocationValue']['arguments']['features']['arrayValue']['values']

            # Convertir cada característica a un objeto ee.Feature
            ee_features = []
            for feature in features:
                geom = feature['functionInvocationValue']['arguments']['geometry']['functionInvocationValue']['arguments']['coordinates']['constantValue']
                if isinstance(geom[0][0], list):
                    ee_geom = ee.Geometry.Polygon(geom)
                else:
                    ee_geom = ee.Geometry.MultiLineString(geom)
                ee_feature = ee.Feature(ee_geom)
                ee_features.append(ee_feature)
            
            # Crear la FeatureCollection
            fc = ee.FeatureCollection(ee_features)
            return fc
        except json.JSONDecodeError as e:
            print("Error al decodificar JSON:", e)
    else:
        print("No se encontró un JSON válido en la cadena.")

In [150]:
from folium import plugins

S1A = ee.ImageCollection("COPERNICUS/S1_GRD")\
.filterDate("2021-05-12", "2021-05-15")\
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', "HH"))

band = ["HH", "angle"]

img = S1A.select(band).mean()

# Crear un mapa centrado en el primer punto
m = geemap.Map(center=[iceberg1['lat'].iloc[0], iceberg2['lon'].iloc[0]], zoom=4)
vis_seg = {'bands': ["HH"], 'min': -30, 'max': 0}
m.addLayer(img, vis_seg, 'sat')

# Añadir líneas al mapa
coordinates = list(zip(iceberg1['lat'], iceberg1['lon']))
m.add_points_from_xy(iceberg1, x='lon', y='lat', popup=['fecha', 'similaridad', 'area'], layer_name="Puntos del iceberg")
line_string = ee.Geometry.LineString(coordinates)
m.addLayer(ee.FeatureCollection(ee.Feature(line_string)), {'color': 'blue'}, 'Línea de movimiento')


polygon_style = {'color': 'red', 'fillColor': 'red', 'fillOpacity': 0.5}
j=0
# Añadir marcadores para cada punto
for i, row in iceberg1.iterrows():
    m.add_marker(location=[row['lat'], row['lon']])
    if i == j + 5:
        variable = trans(row['poligono'])
        geojson_data = variable.getInfo()
        folium.GeoJson(geojson_data).add_to(m)
        j = i


# Guardar el mapa en un archivo HTML
m.save("iceberg_movimiento.html")

JSON extraído: {
  "functionInvocationValue": {
    "functionName": "Collection",
    "arguments": {
      "features": {
        "arrayValue": {
          "values": [
            {
              "functionInvocationValue": {
                "functionName": "Feature",
                "arguments": {
                  "geometry": {
                    "functionInvocationValue": {
                      "functionName": "GeometryConstructors.Polygon",
                      "arguments": {
                        "coordinates": {
                          "constantValue": [
                            [
                              [
                                -106.05618042149165,
                                -74.42542128930235
                              ],
                              [
                                -106.05725839983259,
                                -74.42542128930235
                              ],
                              [
                            

AttributeError: 'Map' object has no attribute 'add_child'

In [143]:
m = folium.Map(location=[iceberg1['lat'].iloc[0], iceberg1['lon'].iloc[0]], zoom_start=4)

# Añadir puntos al mapa y conectar los puntos con líneas
coordinates1 = []
coordinates2 = []

for i, row in iceberg1.iterrows():
    coordinates1.append([row['lat'], row['lon']])
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"Fecha: {row['fecha'].strftime('%Y-%m-%d')}<br>Similitud: {row['similaridad']}<br>Área: {row['area']}"
    ).add_to(m)

j=0
for i, row in iceberg2.iterrows():
    coordinates2.append([row['lat'], row['lon']])
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"Fecha: {row['fecha'].strftime('%Y-%m-%d')}<br>Similitud: {row['similaridad']}<br>Área: {row['area']}"
    ).add_to(m)


# Añadir líneas que conectan los puntos
folium.PolyLine(coordinates1, color="blue", weight=2.5, opacity=1).add_to(m)
folium.PolyLine(coordinates2, color="red", weight=2.5, opacity=1).add_to(m)



# Guardar el mapa en un archivo HTML
m.save("iceberg_movimiento.html")


In [156]:
S1A = ee.ImageCollection("COPERNICUS/S1_GRD")\
.filterDate("2021-05-12", "2021-05-13")\
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', "HH"))

band = ["HH", "angle"]

img = S1A.select(band).mean()

Map = geemap.Map(center=[iceberg1['lat'].iloc[0], iceberg1['lon'].iloc[0]], zoom = 10)
vis_seg = {'bands': ["HH"], 'min': -30, 'max': 0}
Map.addLayer(img, vis_seg, 'sat')

polygon_style1 = {'color': 'red', 'fillColor': 'red', 'fillOpacity': 0.5}
point_style1 = {'color': 'blue', 'pointSize': 5}
polygon_style2 = {'color': 'blue', 'fillColor': 'blue', 'fillOpacity': 0.5}
point_style2 = {'color': 'black', 'pointSize': 5}
j=0
for i, row in iceberg1.iterrows():
    if i == j + 5:
        variable = trans(row.poligono)
        Map.addLayer(variable, polygon_style1, f'ice1{i}')
        centroide_ee = ee.Geometry.Point(row['centroide'])
        j = i
    Map.addLayer(centroide_ee, point_style1, f'cent-ice1{i}')

j=0
for i, row in iceberg2.iterrows():
    if i == j + 5:
        variable = trans(row.poligono)
        Map.addLayer(variable, polygon_style2, f'ice2{i}')
        centroide_ee = ee.Geometry.Point(row['centroide'])
        j = i
    Map.addLayer(centroide_ee, point_style2, f'cent-ice2{i}')

Map.addLayerControl()
Map.save("iceberg_movimiento.html")

In [157]:
acotado1 = iceberg1.head()
acotado2 = iceberg2.head()

In [159]:
S1A = ee.ImageCollection("COPERNICUS/S1_GRD")\
.filterDate("2021-05-12", "2021-05-13")\
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', "HH"))

band = ["HH", "angle"]

img = S1A.select(band).mean()

Map = geemap.Map(center=[acotado1['lat'].iloc[0], acotado1['lon'].iloc[0]], zoom = 10)
vis_seg = {'bands': ["HH"], 'min': -30, 'max': 0}
Map.addLayer(img, vis_seg, 'sat')

polygon_style1 = {'color': 'red', 'fillColor': 'red', 'fillOpacity': 0.5}
point_style1 = {'color': 'blue', 'pointSize': 5}
polygon_style2 = {'color': 'blue', 'fillColor': 'blue', 'fillOpacity': 0.5}
point_style2 = {'color': 'black', 'pointSize': 5}

for i, row in acotado1.iterrows():
    variable = trans(row.poligono)
    Map.addLayer(variable, polygon_style1, f'ice1{i}')
    centroide_ee = ee.Geometry.Point(row['centroide'])
    Map.addLayer(centroide_ee, point_style1, f'cent-ice1{i}')


for i, row in acotado2.iterrows():
    variable = trans(row.poligono)
    Map.addLayer(variable, polygon_style2, f'ice2{i}')
    centroide_ee = ee.Geometry.Point(row['centroide'])
    Map.addLayer(centroide_ee, point_style2, f'cent-ice2{i}')

Map.addLayerControl()
Map.save("iceberg_movimiento.html")

In [146]:
import re

def trans(contenido_celda):
    # Eliminar caracteres no válidos antes y después del JSON
    contenido_celda = contenido_celda.strip()

    match = re.search(r'(\{.*\})', contenido_celda, re.DOTALL)
    # Intentar cargar el JSON
    if match:
        json_str = match.group(1)
        print("JSON extraído:", json_str)
        
        # Intentar cargar el JSON
        try:
            json_dict  = json.loads(json_str)
            print("JSON cargado correctamente.")

            features = json_dict['functionInvocationValue']['arguments']['features']['arrayValue']['values']

            # Convertir cada característica a un objeto ee.Feature
            ee_features = []
            for feature in features:
                geom = feature['functionInvocationValue']['arguments']['geometry']['functionInvocationValue']['arguments']['coordinates']['constantValue']
                if isinstance(geom[0][0], list):
                    ee_geom = ee.Geometry.Polygon(geom)
                else:
                    ee_geom = ee.Geometry.MultiLineString(geom)
                ee_feature = ee.Feature(ee_geom)
                ee_features.append(ee_feature)
            
            # Crear la FeatureCollection
            fc = ee.FeatureCollection(ee_features)
            return fc
        except json.JSONDecodeError as e:
            print("Error al decodificar JSON:", e)
    else:
        print("No se encontró un JSON válido en la cadena.")

In [147]:
a = trans(iceberg1['poligono'][0])

JSON extraído: {
  "functionInvocationValue": {
    "functionName": "Collection",
    "arguments": {
      "features": {
        "arrayValue": {
          "values": [
            {
              "functionInvocationValue": {
                "functionName": "Feature",
                "arguments": {
                  "geometry": {
                    "functionInvocationValue": {
                      "functionName": "GeometryConstructors.Polygon",
                      "arguments": {
                        "coordinates": {
                          "constantValue": [
                            [
                              [
                                -105.83303890491636,
                                -74.52854788391927
                              ],
                              [
                                -105.83052362212082,
                                -74.52854788391927
                              ],
                              [
                            